In [1]:
import pickle

import pandas as pd
import scipy.sparse
import scipy.sparse
from sklearn import preprocessing

from lightfm import LightFM

In [2]:
df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-books/5-core/train.parquet')
test_df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-books/5-core/test.parquet')
categories = pd.read_parquet('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Books_categories.parquet')
categories.category_1 = categories.category_1.map(lambda s: s.replace('&amp;', '&'))

In [3]:
interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/books_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/books_test_interactions.npz')
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/books_dataset.pkl')

In [4]:
items = df['asin'].drop_duplicates()
item_mapping = dataset.mapping()[2]
items_categories = categories[categories.asin.isin(items)].drop_duplicates()
without_category = set(items.values) - set(items_categories.asin)

In [5]:
len(without_category)

11297

In [6]:
le = preprocessing.LabelEncoder()
unique_categories = items_categories.category_1.unique()
le.fit(unique_categories)

LabelEncoder()

In [7]:
to_add = pd.Series([item for item in list(without_category)], name='asin')
to_add = pd.DataFrame(to_add)
to_add['category_1'] = -1

In [8]:
items_categories.category_1 = le.transform(items_categories.category_1)
items_categories = pd.concat([items_categories, to_add])
items_categories.index = items_categories.asin.map(lambda x: item_mapping[x])
items_categories = items_categories.sort_index()
item_category_mapping = items_categories.category_1

In [9]:
item_category_mapping

asin
0          1
1         19
2         19
3         22
4         19
          ..
382647     5
382648    17
382649    30
382650    22
382651    -1
Name: category_1, Length: 382652, dtype: int64

In [10]:
model = LightFM(no_components=100, loss='warp', learning_schedule='adadelta', num_categories=len(unique_categories), item_category_mapping=item_category_mapping)

my new implementation


In [11]:
num_epochs = [5, 25, 50, 250, 500, 1000]
remaining = [num_epochs[0]] + [num_epochs[i] - num_epochs[i-1] for i in range(1, len(num_epochs))]
for i, epochs in enumerate(num_epochs):
    model.fit_partial(interactions, verbose=True, epochs=remaining[i], num_threads=8)
    pickle.dump(model, open(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_epochs_all_clusters_books.pkl', 'wb'), protocol=4)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

category a:  10 category_b:  24


Epoch:  20%|██        | 1/5 [00:23<01:33, 23.44s/it]

category a:  16 category_b:  21


Epoch:  40%|████      | 2/5 [00:46<01:10, 23.38s/it]

...

Epoch: 100%|█████████▉| 498/500 [1:53:53<00:25, 12.92s/it]

category a:  29 category_b:  7


Epoch: 100%|█████████▉| 499/500 [1:54:06<00:13, 13.10s/it]

category a:  10 category_b:  0


Epoch: 100%|██████████| 500/500 [1:54:22<00:00, 13.73s/it]
